# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [3]:
from pprint import pprint
from pathlib import Path

import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [5]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Get the path and file name pattern from the simulation file naming convention
events_fname = Path(sim.env.events_log_fname)

# Get the path to the file which is one level up from the logging outputs
fpath = events_fname.parents[1]

# Get the filename and update it to the correct name
fname = events_fname.name
fname = fname.replace("_events.log", "_metrics_inputs.yaml")

# Load the metrics data
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

False


## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 92.2%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.91,0.95,0.98,0.97,0.95,0.97,0.97,0.97,0.97,0.91,0.84,0.96,0.92,0.88,0.98,0.97,0.97,0.97,0.98,0.98,0.90,0.97,0.97,0.97,0.98,0.92,0.86,0.97,0.91,0.83,0.98,0.97,0.97,0.50,0.96,0.91,0.97,0.96,0.97,0.97,0.97,0.96,0.97,0.97,0.93,0.27,0.98,0.93,0.92,0.97,0.98,0.75,0.97,0.94,0.97,0.81,0.97,0.98,0.98,0.97,0.98,0.97,0.82,0.97,0.97,0.97,0.85,0.97,0.97,0.83,0.89,0.91,0.97,0.93,0.84,0.85,0.98,0.97,0.81,0.74


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,year,windfarm
0,2003,0.97
1,2004,0.95
2,2005,0.95
3,2006,0.94
4,2007,0.94
5,2008,0.87
6,2009,0.86
7,2010,0.90
8,2011,0.91
9,2012,0.92


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,month,windfarm
0,1,0.92
1,2,0.92
2,3,0.91
3,4,0.91
4,5,0.92
5,6,0.92
6,7,0.93
7,8,0.93
8,9,0.93
9,10,0.93


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

,year,month,windfarm
0,2003,1,0.98
1,2003,2,0.98
2,2003,3,0.99
3,2003,4,0.99
4,2003,5,0.98
5,2003,6,0.96
6,2003,7,0.97
7,2003,8,0.98
8,2003,9,0.98
9,2003,10,0.96


## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.44%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.43,0.45,0.46,0.46,0.45,0.47,0.46,0.47,0.46,0.43,0.41,0.46,0.44,0.41,0.47,0.47,0.46,0.46,0.47,0.47,0.43,0.46,0.46,0.46,0.46,0.44,0.40,0.46,0.43,0.40,0.47,0.46,0.46,0.23,0.46,0.43,0.46,0.46,0.46,0.46,0.46,0.46,0.46,0.46,0.44,0.12,0.47,0.44,0.43,0.46,0.47,0.36,0.46,0.45,0.47,0.38,0.46,0.46,0.47,0.46,0.47,0.46,0.39,0.46,0.46,0.46,0.41,0.47,0.46,0.40,0.42,0.44,0.46,0.45,0.40,0.41,0.46,0.46,0.38,0.35


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.44
2005,0.47
2006,0.44
2007,0.47
2008,0.45
2009,0.41
2010,0.38
2011,0.46


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.58
2,0.48
3,0.48
4,0.39
5,0.37
6,0.31
7,0.32
8,0.35
9,0.44


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.62
     2          0.36
     3          0.38
     4          0.53
     5          0.35
     6          0.30
     7          0.30
     8          0.30
     9          0.27
     10         0.42
     11         0.54
     12         0.54
2004 1          0.54
     2          0.50
     3          0.50
     4          0.38
     5          0.29
     6          0.37
     7          0.26
     8          0.40
     9          0.57
     10         0.56
     11         0.48
     12         0.48

## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 72%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 98%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,year,Completion Rate
0,2003,0.97
1,2004,0.91
2,2005,0.96
3,2006,0.99
4,2007,0.98
5,2008,1.00
6,2009,1.01
7,2010,1.01
8,2011,0.99
9,2012,1.03


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,year,Completion Rate
0,1,0.89
1,2,0.96
2,3,1.05
3,4,1.01
4,5,1.05
5,6,1.01
6,7,1.08
7,8,0.98
8,9,0.94
9,10,0.99


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

,year,month,Completion Rate
0,2003,1,0.88
1,2003,2,1.01
2,2003,3,1.06
3,2003,4,1.00
4,2003,5,0.95
5,2003,6,1.06
6,2003,7,1.02
7,2003,8,0.96
8,2003,9,0.92
9,2003,10,1.00


## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $527,620.18/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","2,660,000.00","104,789,062.50"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,year,equipment_cost
0,2003,"12,674,406.25"
1,2004,"13,729,687.50"
2,2005,"11,922,875.00"
3,2006,"13,874,437.50"
4,2007,"11,102,562.50"
5,2008,"11,629,687.50"
6,2009,"12,905,687.50"
7,2010,"13,205,687.50"
8,2011,"13,574,437.50"
9,2012,"12,009,375.00"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,month,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1,"543,010.42","543,010.42","543,010.42",0.00,0.00
1,2,"495,250.00","495,250.00","495,250.00",0.00,0.00
2,3,"542,500.00","542,500.00","542,500.00",0.00,0.00
3,4,"525,000.00","525,000.00","525,000.00",0.00,0.00
4,5,"542,500.00","542,500.00","542,500.00",0.00,"35,207,812.50"
5,6,"525,000.00","525,000.00","525,000.00","2,660,000.00",0.00
6,7,"542,500.00","542,500.00","542,500.00",0.00,"33,104,687.50"
7,8,"542,500.00","542,500.00","542,500.00",0.00,0.00
8,9,"525,000.00","525,000.00","525,000.00",0.00,"36,025,000.00"
9,10,"542,500.00","542,500.00","542,500.00",0.00,"451,562.50"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

,year,month,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,2003,1,"54,760.42","54,760.42","54,760.42",0.00,0.00
1,2003,2,"49,000.00","49,000.00","49,000.00",0.00,0.00
2,2003,3,"54,250.00","54,250.00","54,250.00",0.00,0.00
3,2003,4,"52,500.00","52,500.00","52,500.00",0.00,0.00
4,2003,5,"54,250.00","54,250.00","54,250.00",0.00,"2,750,000.00"
5,2003,6,"52,500.00","52,500.00","52,500.00","266,000.00",0.00
6,2003,7,"54,250.00","54,250.00","54,250.00",0.00,"3,570,312.50"
7,2003,8,"54,250.00","54,250.00","54,250.00",0.00,0.00
8,2003,9,"52,500.00","52,500.00","52,500.00",0.00,"4,170,312.50"
9,2003,10,"54,250.00","54,250.00","54,250.00",0.00,0.00


## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,0.97,0.97,0.97,0.93,0.96


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,0.67,0.67,0.67,0.31,0.65
2004,1.00,1.00,1.00,1.00,1.00
2005,1.00,1.00,1.00,1.00,1.00
2006,1.00,1.00,1.00,1.00,1.00
2007,1.00,1.00,1.00,1.00,1.00
2008,1.00,1.00,1.00,1.00,1.00
2009,1.00,1.00,1.00,1.00,1.00
2010,1.00,1.00,1.00,1.00,1.00
2011,1.00,1.00,1.00,1.00,1.00


## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [24]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [25]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [26]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,year,total_labor_cost
0,2003,0.00
1,2004,0.00
2,2005,0.00
3,2006,0.00
4,2007,0.00
5,2008,0.00
6,2009,0.00
7,2010,0.00
8,2011,0.00
9,2012,0.00


In [27]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,month,total_labor_cost
0,1,0.00
1,2,0.00
2,3,0.00
3,4,0.00
4,5,0.00
5,6,0.00
6,7,0.00
7,8,0.00
8,9,0.00
9,10,0.00


In [28]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

,year,month,total_labor_cost
0,2003,1,0.00
1,2003,2,0.00
2,2003,3,0.00
3,2003,4,0.00
4,2003,5,0.00
5,2003,6,0.00
6,2003,7,0.00
7,2003,8,0.00
8,2003,9,0.00
9,2003,10,0.00


## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [29]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,reason,total_cost
0,Maintenance,"1,611,458.33"
1,Repair,"24,354,411.46"
2,Mobilization,"15,000,000.00"
3,Weather Delay,"15,572,447.92"
4,No Requests,"9,446,739.58"
5,Not in Shift,"30,884,192.71"
6,NaN,"28,357,125.00"


In [30]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,reason,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
0,Maintenance,0.00,0.00,0.00,"1,611,458.33","1,611,458.33"
1,Repair,0.00,0.00,0.00,"24,354,411.46","24,354,411.46"
2,Mobilization,0.00,0.00,0.00,"15,000,000.00","15,000,000.00"
3,Weather Delay,0.00,0.00,0.00,"15,572,447.92","15,572,447.92"
4,No Requests,0.00,0.00,0.00,"9,446,739.58","9,446,739.58"
5,Not in Shift,0.00,0.00,0.00,"30,884,192.71","30,884,192.71"
6,NaN,0.00,0.00,0.00,"28,357,125.00","28,357,125.00"


In [31]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

,year,reason,total_cost
0,2003,Maintenance,"28,000.00"
1,2003,Repair,"2,091,091.15"
2,2003,Mobilization,"1,500,000.00"
3,2003,Weather Delay,"934,205.73"
4,2003,No Requests,"3,901,343.75"
5,2003,Not in Shift,"2,227,322.92"
6,2003,NaN,"1,882,125.00"
7,2004,Maintenance,"126,091.15"
8,2004,Repair,"2,903,734.38"
9,2004,Mobilization,"1,500,000.00"


In [32]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

,month,reason,total_cost
0,1,Maintenance,"87,445.31"
1,1,Repair,"504,255.21"
2,1,Mobilization,0.00
3,1,Weather Delay,"156,315.10"
4,1,No Requests,"29,440.10"
5,1,Not in Shift,"135,260.42"
6,1,NaN,"686,000.00"
7,2,Maintenance,"99,330.73"
8,2,Repair,"465,044.27"
9,2,Mobilization,0.00


In [33]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

,year,month,reason,total_cost
0,2003,1,Maintenance,0.00
1,2003,1,Repair,"52,007.81"
2,2003,1,Mobilization,0.00
3,2003,1,Weather Delay,"15,640.62"
4,2003,1,No Requests,"29,440.10"
5,2003,1,Not in Shift,"5,122.40"
6,2003,1,NaN,"59,500.00"
7,2003,2,Maintenance,0.00
8,2003,2,Repair,"57,221.35"
9,2003,2,Mobilization,0.00


## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
##### **NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [34]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,component,total_cost
0,cable,0.00
1,drive_train,0.00
2,electrical_system,0.00
3,electronic_control,0.00
4,gearbox,0.00
5,generator,"134,255,572.92"
6,hydraulic_system,0.00
7,mechanical_brake,0.00
8,rotor_blades,0.00
9,rotor_hub,0.00


In [35]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

,component,action,materials_cost,total_labor_cost,equipment_cost,total_cost
0,generator,delay,0,0.00,"74,812,234.38","74,812,234.38"
1,generator,maintenance,0,0.00,"1,611,458.33","1,611,458.33"
2,generator,repair,0,0.00,"24,354,411.46","24,354,411.46"


In [36]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,year,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,2003,cable,0.00,0.00,0.00,0.00
1,2003,drive_train,0.00,0.00,0.00,0.00
2,2003,electrical_system,0.00,0.00,0.00,0.00
3,2003,electronic_control,0.00,0.00,0.00,0.00
4,2003,gearbox,0.00,0.00,0.00,0.00
5,2003,generator,"2,261,500.00",0.00,"7,271,531.25","9,533,031.25"
6,2003,hydraulic_system,0.00,0.00,0.00,0.00
7,2003,mechanical_brake,0.00,0.00,0.00,0.00
8,2003,rotor_blades,0.00,0.00,0.00,0.00
9,2003,rotor_hub,0.00,0.00,0.00,0.00


In [37]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,month,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,1,cable,0.00,0.00,0.00,0.00
1,1,drive_train,0.00,0.00,0.00,0.00
2,1,electrical_system,0.00,0.00,0.00,0.00
3,1,electronic_control,0.00,0.00,0.00,0.00
4,1,gearbox,0.00,0.00,0.00,0.00
5,1,generator,"824,500.00",0.00,"1,598,059.90","2,422,559.90"
6,1,hydraulic_system,0.00,0.00,0.00,0.00
7,1,mechanical_brake,0.00,0.00,0.00,0.00
8,1,rotor_blades,0.00,0.00,0.00,0.00
9,1,rotor_hub,0.00,0.00,0.00,0.00


In [38]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

,year,month,component,materials_cost,total_labor_cost,equipment_cost,total_cost
0,2003,1,cable,0.00,0.00,0.00,0.00
1,2003,1,drive_train,0.00,0.00,0.00,0.00
2,2003,1,electrical_system,0.00,0.00,0.00,0.00
3,2003,1,electronic_control,0.00,0.00,0.00,0.00
4,2003,1,gearbox,0.00,0.00,0.00,0.00
5,2003,1,generator,"73,500.00",0.00,"133,309.90","206,809.90"
6,2003,1,hydraulic_system,0.00,0.00,0.00,0.00
7,2003,1,mechanical_brake,0.00,0.00,0.00,0.00
8,2003,1,rotor_blades,0.00,0.00,0.00,0.00
9,2003,1,rotor_hub,0.00,0.00,0.00,0.00


## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [39]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [40]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,000,080.00"


In [41]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [42]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [43]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,year,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
1,2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2,2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
3,2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
4,2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
5,2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
6,2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
7,2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
8,2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
9,2012,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [44]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"8,018,335.00","58,078.50","50,478.50",479.00
major repair,"86,269.85","1,735.50","1,283.00",25.00
major replacement,"266,288.98","13,248.25","266,288.98",55.00
manual reset,"674,693.33","136,279.00","13,946.00","5,485.00"
medium repair,"14,360.37","10,616.25","6,518.25",197.00
minor repair,"105,901.27","51,374.25","5,041.25","2,166.00"


## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [45]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,243,073,825.00"


In [46]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,243,073,825.00","112,921,844.50","119,122,868.00","122,140,588.00","121,693,579.50","119,150,778.00","122,461,996.00","121,438,348.50","122,788,210.00","122,075,081.00","112,066,320.50","106,561,947.50","120,859,093.00","114,674,063.50","108,694,595.00","122,838,477.00","122,527,088.00","121,796,389.00","121,480,114.50","122,566,049.50","122,857,969.50","113,771,162.00","121,174,467.00","121,661,621.00","122,148,112.50","122,253,948.50","114,945,349.00","105,094,032.00","121,525,325.50","112,093,241.00","104,006,649.00","122,702,691.50","120,861,613.00","121,361,828.00","60,975,470.50","121,238,857.50","112,840,164.00","121,914,350.00","121,282,611.00","122,075,546.00","122,179,948.50","121,551,836.00","120,301,004.50","121,223,852.50","121,846,491.00","116,310,757.00","31,054,377.50","122,654,480.00","114,938,915.50","113,508,547.00","121,715,543.00","122,642,766.50","94,245,073.00","121,738,622.00","118,639,821.50","122,322,816.50","101,017,087.50","122,214,102.50","122,273,062.00","122,801,626.00","121,019,643.50","122,993,105.00","121,777,092.50","101,735,029.00","121,695,713.50","121,559,799.00","121,177,607.00","107,433,392.50","122,533,681.00","122,040,953.00","106,444,746.50","110,372,320.00","114,622,195.00","120,989,579.00","117,465,194.50","105,629,920.50","107,066,062.00","122,227,323.00","121,745,281.00","101,035,568.50","91,686,449.50"


In [47]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"860,996,882.00"
2004,"935,447,415.50"
2005,"977,923,142.50"
2006,"921,258,806.50"
2007,"997,697,074.50"
2008,"958,414,317.50"
2009,"858,855,120.50"
2010,"808,569,186.00"
2011,"967,305,688.50"


In [48]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,031,812,205.50"
2,"780,993,440.00"
3,"854,174,167.50"
4,"674,939,759.50"
5,"656,235,890.00"
6,"532,292,919.50"
7,"565,055,191.50"
8,"619,999,510.50"
9,"767,222,649.00"


In [49]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     110,540,361.50
     2      58,463,112.50
     3      67,051,118.00
     4      92,321,268.50
     5      61,685,378.50
     6      52,003,339.50
     7      52,680,389.50
     8      53,048,137.50
     9      47,293,241.50
     10     75,503,848.50
     11     93,520,627.50
     12     96,886,059.00
2004 1      95,635,868.50
     2      83,688,248.00
     3      88,719,645.00
     4      65,986,923.00
     5      51,431,913.00
     6      64,292,847.00
     7      47,292,117.50
     8      71,360,461.50
     9      98,376,223.50
     10     99,860,215.00
     11     82,792,735.00
     12     86,010,218.50

## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [50]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [51]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [52]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [53]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [54]:
SAM_settings = "SAM_Singleowner_defaults.yaml"
metrics.sam_settings = load_yaml(sim.env.data_dir / "windfarm", SAM_settings)
metrics._setup_pysam()

In [55]:
metrics.pysam_all_outputs()

,Value
Metric,
After Tax NPV ($),"28,393,849.90"
Nominal LCOE ($/kW),-0.00
Real LCOE ($/kW),-0.00
After Tax IRR (%),13.75


In [56]:
sim.env.cleanup_log_files(log_only=False)